In [11]:
import pandas as pd
import geopandas as gpd
import os

# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo'

# Rutas a los tres archivos finales que hemos preparado
path_geometrias = os.path.join(BASE_DIR, '05_censo', 'SECCION.shp')
path_electoral = os.path.join(BASE_DIR, '05_analisis_agregado', 'analisis_agregado_por_seccion.csv')
path_censo = os.path.join(BASE_DIR, '05_censo', 'censo_manzanillo_completo_2020.csv')

# Ruta de salida para el GeoPackage
output_folder = os.path.join(BASE_DIR, '06_geopackage')
output_file = 'Manzanillo_Analisis_Electoral_Censal_Completo.gpkg'

# --- 2. CARGA DE LAS TRES FUENTES DE DATOS ---
print("Cargando las tres fuentes de datos finales...")
gdf_secciones = gpd.read_file(path_geometrias)
df_electoral = pd.read_csv(path_electoral)
df_censo = pd.read_csv(path_censo)
print("¡Datos cargados exitosamente!")

# --- 3. PREPARACIÓN FINAL Y UNIÓN (MERGE) ---
# Asegurarnos que las llaves de unión sean del mismo tipo (int)
gdf_secciones['SECCION'] = gdf_secciones['SECCION'].astype(int)
df_electoral['seccion'] = df_electoral['seccion'].astype(int)
df_censo['seccion'] = df_censo['seccion'].astype(int)

# Primera unión: Geometrías con datos electorales agregados
print("Uniendo geometrías con datos electorales...")
# Usamos solo las columnas de geometría y la llave del GeoDataFrame para evitar duplicados
gdf_merged = gdf_secciones[['SECCION', 'geometry']].merge(df_electoral, left_on='SECCION', right_on='seccion', how='left')

# Segunda unión: El resultado anterior con TODOS los datos del censo
print("Uniendo el resultado con los datos del censo...")
gdf_final = gdf_merged.merge(df_censo, on='seccion', how='left')

# Eliminar columnas redundantes si las hay
if 'SECCION' in gdf_final.columns:
    gdf_final = gdf_final.drop(columns=['SECCION'])

# --- 4. GUARDADO DEL GEOPACKAGE ---
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)

# Usamos el driver 'GPKG' para crear el GeoPackage
gdf_final.to_file(output_path, driver='GPKG')

print("\n\n¡PROYECTO FINALIZADO!")
print(f"GeoPackage con censo completo creado exitosamente en: {output_path}")
print("\n--- Resumen del GeoDataFrame Final ---")
gdf_final.info()

Cargando las tres fuentes de datos finales...
¡Datos cargados exitosamente!
Uniendo geometrías con datos electorales...
Uniendo el resultado con los datos del censo...


¡PROYECTO FINALIZADO!
GeoPackage con censo completo creado exitosamente en: /Users/omartellez/Manzanillo/06_geopackage/Manzanillo_Analisis_Electoral_Censal_Completo.gpkg

--- Resumen del GeoDataFrame Final ---
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5535 entries, 0 to 5534
Columns: 234 entries, geometry to VPH_SINTIC
dtypes: float64(232), geometry(1), object(1)
memory usage: 9.9+ MB
